In [15]:
import struct
import numpy as np
from array import array
import matplotlib.pyplot as plt
import math
import numpy.linalg as LA
import pandas as pandas


def readExcelSheet1(excelfile):
    from pandas import read_excel
    return (read_excel(excelfile)).values


def readExcelRange(excelfile,sheetname="Data",startrow=1,endrow=1,startcol=1,endcol=1):
    from pandas import read_excel
    values=(read_excel(excelfile, sheetname,header=None)).values;
    return values[startrow-1:endrow,startcol-1:endcol]

def readExcel(excelfile,**args):
    if args:
        data=readExcelRange(excelfile,**args)
    else:
        data=readExcelSheet1(excelfile)
    if data.shape==(1,1):
        return data[0,0]
    elif (data.shape)[0]==1:
        return data[0]
    else:
        return data

    
def writeExcelData(x,excelfile,sheetname,startrow,startcol):
    from pandas import DataFrame, ExcelWriter
    from openpyxl import load_workbook
    df=DataFrame(x)
    book = load_workbook(excelfile)
    writer = ExcelWriter(excelfile, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, sheet_name=sheetname,startrow=startrow-1, startcol=startcol-1, header=False, index=False)
    writer.save()
    writer.close()


def getSheetNames(excelfile):
    from pandas import ExcelFile
    return (ExcelFile(excelfile)).sheet_names
    

# excelfile string of path to .xlsx file
excelfile = "Assignment_4_Data_and_Template.xlsx"





sheets = getSheetNames(excelfile) # array of sheet names
#print(sheets)



data=readExcel(excelfile) # array of rows (row is array of read data from sheet) of excel file
#print(data.shape)
#print(data[0])

X = data[:,0:15] # dim(X) = 6600 x 15 (Nxd)
#print(X[0])
#print(X.shape)

T = data[:,15:16] #dim(T) = 6600 x 2 (NxC)
#print(T[0])
#print(T.shape)



ones = np.ones(len(X)).reshape(len(X),1)
#print(ones.shape)


Xa = np.concatenate((ones, X), axis=1) # dim(Xa) = 6600 x 16, aka: N x (d+1) 
#print(Xa)
#print(Xa.shape)


W = (np.linalg.pinv(Xa)).dot(T) # dim(W) = 16 x 2, aka: (d+1)xc
#print(W.shape)
#print(W)



#print(Xa[199].dot(W))
#print(T[199])

types = data[:,16:17]
#print(types)
T6 = np.ones(len(types)*6).reshape(len(types),6)
T6 = T6*-1
#print(T6)

#zero = np.zeros(6); zero[0] = 1
#one = np.zeros(6); one[1] = 1
#two = np.zeros(6); two[2] = 1
#three = np.zeros(6); three[3] = 1
#four = np.zeros(6); four[4] = 1
#five = np.zeros(6); five[5] = 1
#print(zero)
#print(one)
#print(two)
#print(three)
#print(four)
#print(five)

#zero = np.ones(6); zero = zero*-1
#one = np.zeros(6); one = one*-1
#two = np.zeros(6); two = two*-1
#three = np.zeros(6); three = three*-1
#four = np.zeros(6); four = four*-1
#five = np.zeros(6); five = five*-1
#print(zero)
#print(one)
#print(two)
#print(three)
#print(four)
#print(five)

for i,b in enumerate(types):
#    if b[0] == 0: T6[i] = zero
#    elif b[0] == 1: T6[i] = one
#    elif b[0] == 2: T6[i] = two
#    elif b[0] == 3: T6[i] = three
#    elif b[0] == 4: T6[i] = four
#    elif b[0] == 5: T6[i] = five
    T6[i][b[0]] = T6[i][b[0]]*-1

#print(T6)




W6 = (np.linalg.pinv(Xa)).dot(T6) # dim(W) = 16 x 6, aka: (d+1)xc
#print(Xa.shape)
#print(T6.shape)
#print(W6.shape)





excelfile2 = "dataPredict.xlsx"

data2=readExcel(excelfile2) # array of rows (row is array of read data from sheet) of excel file
#print(data2.shape)
#print(data2[0])



ones2 = np.ones(len(data2)).reshape(len(data2),1)
#print(ones.shape)


Xa2 = np.concatenate((ones2, data2), axis=1) # dim(Xa) = 6600 x 16, aka: N x (d+1) 
#print(Xa2)
#print(Xa2.shape)







binPredict = Xa2.dot(W)
#print(binPredict.shape)
#print(binPredict[0])


sixClassPre = Xa2.dot(W6)
#print(sixClassPre.shape)
#print(sixClassPre[10])

sixClassPreResult = np.zeros(50).reshape(50,1)



for i,b in enumerate(sixClassPre):
    typeIndex = b.argmax(axis=0)
    
    if typeIndex == 0: sixClassPreResult[i] = 0
    elif typeIndex == 1: sixClassPreResult[i] = 1
    elif typeIndex == 2: sixClassPreResult[i] = 2
    elif typeIndex == 3: sixClassPreResult[i] = 3
    elif typeIndex == 4: sixClassPreResult[i] = 4
    elif typeIndex == 5: sixClassPreResult[i] = 5


#print(sixClassPreResult)
#vd=sixClassPre[10].argmax(axis=0)
#print(vd)
#print(sixClassPre[10][2])



bClassLabel = T
bGroundTruth = Xa.dot(W)

#confusion matrix TP for true positive, we are counting but adding 0ne
TN = TP = FN = FP = 0
for i,b in enumerate(T):
    if bClassLabel[i] > 0 and bGroundTruth[i] > 0 : TP += 1
    elif bClassLabel[i] < 0 and bGroundTruth[i] < 0 : TN += 1
    elif bClassLabel[i] > 0 and bGroundTruth[i] < 0 : FP += 1
    elif bClassLabel[i] < 0 and bGroundTruth[i] > 0 : FN += 1

print(TN, TP, FN, FP)

bAccuracy = (TN+TP) / (TN+TP+FN+FP)
bSensitivity = TP / (TP+FN)
bSpecificity = TN / (TN+FP)
bPPV = TP / (TP+FP)

print(bAccuracy,bSensitivity,bSpecificity,bPPV)

#test
testo = (TN+TP)
testo

(3229, 2150, 71, 1150)
(0, 0, 0, 0)


5379

In [11]:
sixClassConfusion = np.zeros(36).reshape(6,6)
#print(sixClassConfusion)

#print(T6.shape)

sixGroundTruth = Xa.dot(W6)
#print(sixGroundTruth.shape)



for i,b in enumerate(sixGroundTruth):
    typeIndex = b.argmax(axis=0)
    typeIndexLabel = T6[i].argmax(axis=0)
    sixClassConfusion[typeIndexLabel][typeIndex] +=1
    
    
#print(sixClassConfusion)



ppvSixClass = np.zeros(6)
print(ppvSixClass)

for i,b in enumerate(ppvSixClass):
    ppvSixClass[i] = sixClassConfusion[i][i] / np.sum(sixClassConfusion[:,i])
    
    
print(ppvSixClass)

highestPPV = np.max(ppvSixClass)
highestPPVindex = ppvSixClass.argmax()
lowestPPV = np.min(ppvSixClass)
lowestPPVindex = ppvSixClass.argmin()
print(highestPPV)
print(highestPPVindex)
print(lowestPPV)
print(lowestPPVindex)

[ 0.  0.  0.  0.  0.  0.]
[ 0.71412924  0.47831715  0.68499607  0.33652008  0.99634369  0.57907348]
0.99634369287
4
0.336520076482
3


In [5]:
#writeExcelData(W,excelfile,'Classifiers',5,1) # binary classifier
#writeExcelData(W6,excelfile,'Classifiers',5,5) # 6-class classifier
#writeExcelData(binPredict,excelfile,'To be classified',5,16) # bin classifier on data
writeExcelData(sixClassPreResult,excelfile,'To be classified',5,17) # 6-class classifier on data
writeExcelData([TN],excelfile,'Performance',10,3) # binary TN
writeExcelData([FN],excelfile,'Performance',10,4) # binary FN
writeExcelData([FP],excelfile,'Performance',11,3) # binary FP
writeExcelData([TP],excelfile,'Performance',11,4) # binary TP
writeExcelData([bAccuracy],excelfile,'Performance',8,7) # binary Accuracy
writeExcelData([bSensitivity],excelfile,'Performance',9,7) # binary bSensitivity
writeExcelData([bSpecificity],excelfile,'Performance',10,7) # binary bSensitivity
writeExcelData([bPPV],excelfile,'Performance',11,7) # binary bSensitivity
writeExcelData(sixClassConfusion,excelfile,'Performance',19,3) # true class label
writeExcelData([highestPPV],excelfile,'Performance',20,12) # highestPPV %
writeExcelData([highestPPVindex],excelfile,'Performance',20,13) # highestPPV label
writeExcelData([lowestPPV],excelfile,'Performance',21,12) # lowestPPV %
writeExcelData([lowestPPVindex],excelfile,'Performance',21,13) # lowestPPVindex label

In [6]:
writeExcelData([bAccuracy],excelfile,'Performance',8,7) # binary Accuracy

In [7]:
print(bAccuracy,bSensitivity,bSpecificity,bPPV)

(0, 0, 0, 0)
